In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import pandas as pd
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
class CSIRODataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, is_test=False):
        """
        Dataset for CSIRO Image2Biomass competition.

        Args:
            csv_file (str): Path to the train/test CSV file.
            root_dir (str): Root directory where images are stored.
            transform (callable, optional): Optional transform to be applied on an image.
            is_test (bool): If True, dataset is for test data (no targets).
        """
        self.df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.is_test = is_test

        if not self.is_test:
            self.df = self.df.pivot_table(
                index=["image_path", "Sampling_Date", "State", "Species", "Pre_GSHH_NDVI", "Height_Ave_cm"],
                columns="target_name",
                values="target"
            ).reset_index()
            
            self.target_cols = ["Dry_Clover_g", "Dry_Dead_g", "Dry_Green_g", "Dry_Total_g", "GDM_g"]
        else:
            self.df = self.df.drop_duplicates(subset="image_path").reset_index(drop=True)
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.df.iloc[idx]["image_path"])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        if self.is_test:
            sample_id = self.df.iloc[idx]["image_path"].split("/")[-1].split(".")[0]
            target_name = self.df.iloc[idx]["target_name"]
            return image, sample_id, target_name

        ndvi = torch.tensor(self.df.iloc[idx]["Pre_GSHH_NDVI"], dtype=torch.float32)
        height = torch.tensor(self.df.iloc[idx]["Height_Ave_cm"], dtype=torch.float32)

        state = self.df.iloc[idx]["State"]
        species = self.df.iloc[idx]["Species"]          

        y = torch.tensor(self.df.loc[idx, self.target_cols].values.astype("float32"))

        return {
            "image": image,
            "ndvi": ndvi,
            "height": height,
            "state": state,
            "species": species,
            "target": y,
        }

In [ ]:
# train_transform = transforms.Compose(
#     [
#         transforms.RandomResizedCrop(224),
#         transforms.RandomHorizontalFlip(p=0.5),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
#     ]
# )
# test_transform = transforms.Compose(
#     [
#         transforms.RandomResizedCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
#     ]
# )

from torchvision import transforms
from PIL import ImageFilter

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    
    transforms.RandomRotation(degrees=(0, 360)),
    
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
    
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.2),
    transforms.RandomApply([transforms.Lambda(lambda x: x.filter(ImageFilter.UnsharpMask(radius=2, percent=150)))], p=0.2),

    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])


In [ ]:
train_path = "/kaggle/input/csiro-biomass/train.csv"
test_path = "/kaggle/input/csiro-biomass/test.csv"
train_dir = "/kaggle/input/csiro-biomass/"
test_dir = "/kaggle/input/csiro-biomass/"

In [ ]:
train_ds = CSIRODataset(csv_file=train_path, root_dir=train_dir, transform=train_transform)
test_ds = CSIRODataset(csv_file=test_path, root_dir=test_dir, transform=test_transform, is_test=True)

In [ ]:
train_size = int(0.7 * len(train_ds))
val_size = len(train_ds) - train_size

train_data, val_data = random_split(train_ds, [train_size, val_size])

In [ ]:
train_loader = DataLoader(train_data, batch_size = 32, shuffle=True)
val_loader = DataLoader(val_data, batch_size = 32, shuffle=False)
test_loader = DataLoader(test_ds, batch_size = 32, shuffle=False)

In [ ]:
data_0 = train_ds[0]
print("Label (Numeric):", data_0["target"])

In [ ]:
data_iter = iter(train_loader)
data_r = next(data_iter)
images = data_r["image"]
labels = data_r["target"]

image = images[0].numpy().transpose((1, 2, 0))
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
image = std * image + mean
image = image.clip(0, 1)

plt.imshow(image)
plt.title(f"Label: {labels[0]}")
plt.axis('off')
plt.show()

In [ ]:
import torch.nn as nn
from torchvision import models

class CSIRORegressor(nn.Module):
    def __init__(self, base_model='resnet18', pretrained=False, num_outputs=1):
        super().__init__()
        self.base_model = base_model.lower()

        # -------------------------
        # 🔸 ResNet
        # -------------------------
        if self.base_model == 'resnet18':
            self.backbone = models.resnet18(pretrained=pretrained)
            in_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Linear(in_features, num_outputs)

        elif self.base_model == 'resnet50':
            self.backbone = models.resnet50(pretrained=pretrained)
            in_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Linear(in_features, num_outputs)

        # -------------------------
        # 🔸 EfficientNet
        # -------------------------
        elif self.base_model == 'efficientnet_b0':
            self.backbone = models.efficientnet_b0(pretrained=pretrained)
            in_features = self.backbone.classifier[1].in_features
            self.backbone.classifier[1] = nn.Linear(in_features, num_outputs)

        elif self.base_model == 'efficientnet_b4':
            self.backbone = models.efficientnet_b4(pretrained=pretrained)
            in_features = self.backbone.classifier[1].in_features
            self.backbone.classifier[1] = nn.Linear(in_features, num_outputs)

        # -------------------------
        # 🔸 ConvNeXt
        # -------------------------
        elif self.base_model == 'convnext_tiny':
            self.backbone = models.convnext_tiny(pretrained=pretrained)
            in_features = self.backbone.classifier[2].in_features
            self.backbone.classifier[2] = nn.Linear(in_features, num_outputs)

        elif self.base_model == 'convnext_base':
            self.backbone = models.convnext_base(pretrained=pretrained)
            in_features = self.backbone.classifier[2].in_features
            self.backbone.classifier[2] = nn.Linear(in_features, num_outputs)

        # -------------------------
        # 🔸 ViT (Vision Transformer)
        # -------------------------
        elif self.base_model == 'vit_b_16':
            self.backbone = models.vit_b_16(pretrained=pretrained)
            in_features = self.backbone.heads.head.in_features
            self.backbone.heads.head = nn.Linear(in_features, num_outputs)

        # -------------------------
        # 🔸 DenseNet
        # -------------------------
        elif self.base_model == 'densenet121':
            self.backbone = models.densenet121(pretrained=pretrained)
            in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Linear(in_features, num_outputs)

        else:
            raise ValueError(f"❌ Unknown base model: {self.base_model}")

    def forward(self, x):
        out = self.backbone(x)
        return out.squeeze(1)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = CSIRORegressor(base_model='efficientnet_b0', pretrained=False, num_outputs=5)
model = model.to(device)

In [ ]:
import torch.optim as optim
from sklearn.metrics import r2_score

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=4e-4)

In [ ]:
# weights = {
#     "Dry_Green_g": 0.1,
#     "Dry_Dead_g": 0.1,
#     "Dry_Clover_g": 0.1,
#     "GDM_g": 0.2,
#     "Dry_Total_g": 0.5
# }
weights = {
    "Dry_Clover_g": 0.1,
    "Dry_Dead_g": 0.1,
    "Dry_Green_g": 0.1,
    "Dry_Total_g": 0.5,
    "GDM_g": 0.2
}

In [ ]:
target_names = ["Dry_Clover_g", "Dry_Dead_g", "Dry_Green_g", "Dry_Total_g", "GDM_g"]
# ["Dry_Green_g","Dry_Dead_g","Dry_Clover_g","GDM_g","Dry_Total_g"]

In [ ]:
# def weighted_r2(y_true, y_pred):
#     """
#     Weighted R² for multi-target regression.

#     y_true, y_pred: torch tensors, shape = [num_samples, num_targets]
#     weights: dict, mapping target names to their weights
#     """
#     y_true = y_true.detach().cpu().numpy()
#     y_pred = y_pred.detach().cpu().numpy()

#     w_col = np.array([weights[name] for name in target_names])

#     ss_res = np.sum(((y_true - y_pred)**2) * w_col[np.newaxis, :])
    
#     y_mean = np.sum(y_true * w_col[np.newaxis, :], axis=0) / np.sum(w_col)
    
#     ss_tot = np.sum(((y_true - y_mean[np.newaxis, :])**2) * w_col[np.newaxis, :])

#     return 1 - ss_res / ss_tot


labels = ["Dry_Clover_g", "Dry_Dead_g", "Dry_Green_g", "Dry_Total_g", "GDM_g"]

def weighted_r2(y_true, y_pred) -> float:
    """
    Weighted R² metric for CSIRO Image2Biomass competition.

    Args:
        y_true, y_pred: torch tensors of shape [num_samples, 5]

    Returns:
        float: weighted R² score
    """
    # Convert to numpy
    y_true = y_true.detach().cpu().numpy()
    y_pred = y_pred.detach().cpu().numpy()

    # Weighted mean of targets
    y_weighted = 0
    for l, label in enumerate(labels):
        y_weighted += y_true[:, l].mean() * weights[label]

    # Compute weighted SS_res and SS_tot
    ss_res = 0
    ss_tot = 0
    for l, label in enumerate(labels):
        ss_res += ((y_true[:, l] - y_pred[:, l])**2).mean() * weights[label]
        ss_tot += ((y_true[:, l] - y_weighted)**2).mean() * weights[label]

    return 1 - ss_res / ss_tot



In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 150

for epoch in range(num_epochs):
    # -------------------------
    # 🔹 Training
    # -------------------------
    model.train()
    train_running_loss = 0.0
    train_preds = []
    train_targets = []
    for batch in train_loader:
        images = batch['image'].to(device)
        targets = batch['target'].to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_running_loss += loss.item() * images.size(0)
        train_preds.append(outputs)
        train_targets.append(targets)

    train_loss = train_running_loss / len(train_loader.dataset)
    train_preds = torch.cat(train_preds, dim=0)
    train_targets = torch.cat(train_targets, dim=0)

    train_r2 = weighted_r2(train_targets, train_preds)
    train_mse = mean_squared_error(train_targets.detach().cpu().numpy(), train_preds.detach().cpu().numpy())
    train_mae = mean_absolute_error(train_targets.detach().cpu().numpy(), train_preds.detach().cpu().numpy())

    # -------------------------
    # 🔹 Validation
    # -------------------------
    model.eval()
    val_running_loss = 0.0
    val_preds = []
    val_targets = []
    with torch.no_grad():
        for batch in val_loader:
            images = batch['image'].to(device)
            targets = batch['target'].to(device)

            outputs = model(images)
            loss = criterion(outputs, targets)
            val_running_loss += loss.item() * images.size(0)

            val_preds.append(outputs)
            val_targets.append(targets)

    val_loss = val_running_loss / len(val_loader.dataset)
    val_preds = torch.cat(val_preds, dim=0)
    val_targets = torch.cat(val_targets, dim=0)

    val_r2 = weighted_r2(val_targets, val_preds)
    val_mse = mean_squared_error(val_targets.detach().cpu().numpy(), val_preds.detach().cpu().numpy())
    val_mae = mean_absolute_error(val_targets.detach().cpu().numpy(), val_preds.detach().cpu().numpy())

    # -------------------------
    # 🔹 Print metrics
    # -------------------------
    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"  Train Loss: {train_loss:.4f}, R²: {train_r2:.4f}, MSE: {train_mse:.4f}, MAE: {train_mae:.4f}")
    print(f"  Val   Loss: {val_loss:.4f}, R²: {val_r2:.4f}, MSE: {val_mse:.4f}, MAE: {val_mae:.4f}")
    print("-"*60)


In [ ]:
model.eval()
all_preds = []
sample_ids = []

with torch.no_grad():
    for batch in test_loader:
        images = batch[0].to(device)
        batch_ids = batch[1]
        outputs = model(images)
        outputs = outputs.cpu().numpy()
        
        for i, sid in enumerate(batch_ids):
            for j, tname in enumerate(target_names):
                all_preds.append({
                    "sample_id": f"{sid}__{tname}",
                    "target": float(outputs[i,j])
                })

submission = pd.DataFrame(all_preds)
submission.to_csv("submission.csv", index=False)
print("✅ Submission saved: submission.csv")